# Structured Outputs

**Before you start**, make sure you have:
1. Your Gemini API key from [aistudio.google.com](https://aistudio.google.com)
2. Added it to Colab Secrets (🔑 icon in left sidebar) with the name `GEMINI_API_KEY`

If you already set this up in a previous notebook, the secret should be accessible in this notebook too.

---
## 🔧 Part 1: Setup

Run this cell first to install the library and configure your API key.

In [ ]:
# Install the Google AI library (only needs to run once)
!pip install -q google-genai

# Import and configure
import google.generativeai as genai
from google.colab import userdata
import json
import time

# Get your API key from Colab secrets
genai.configure(api_key=userdata.get('GEMINI_API_KEY'))
print("✅ API key configured successfully!")

## Part 2: Structured Outputs Using Config

Previously, we saw that if we tell a model that we want a response in JSON and how an example of the structure, it seems to work to generate the structured in the correct format. However, this **isn't guaranteed to work every time**.

The Gemini API allows you to specify a `response_schema` in the request as a JSON schema to force the model to always respond with an output that matches it.

### 1. Sentiment Analysis

Use structured outputs to get a sentiment for an input and a confidence about it.

Run the call several times to see if the confidence is non-deterministic!

In [ ]:
review = "Worst pizza I've ever tasted"

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal

model = genai.GenerativeModel("gemini-2.5-flash-lite")

# Defining a pydantic model to create the response schema
# This model is converted into a JSON schema model
class Sentiment(BaseModel):
  sentiment: Literal["positive", "neutral", "negative"]
  confidence: float


response = model.generate_content(review, generation_config=genai.GenerationConfig(
    response_mime_type="application/json",
    response_schema=Sentiment,
))

# Parsing the response into a pydantic model instance
sentiment = Sentiment.model_validate_json(response.text)

print("Sentiment:", sentiment.sentiment)
print("Confidence:", sentiment.confidence)

### 2. Classification

Provide a classification based on a schema

In [ ]:
from io import BytesIO
import requests
from PIL import Image

url = "https://www.thesun.ie/wp-content/uploads/sites/3/2023/09/dunens1.png"
response = requests.get(url)
img = Image.open(BytesIO(response.content))
img

In [ ]:
# Uncomment this to try it with your own images
#
# from google.colab import files
# import PIL

# uploaded = files.upload()

# filename = list(uploaded.keys())[0]
# img = PIL.Image.open(filename)

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal

model = genai.GenerativeModel('gemini-2.5-flash')

class DocumentType(BaseModel):
  document_type: Literal["receipt", "invoice", "bank_statement", "utility_bill", "unknown"]
  confidence: float

response = model.generate_content(["Classify this image", img], generation_config=genai.GenerationConfig(
    response_mime_type="application/json",
    response_schema=DocumentType,
))

# Parsing the response into a pydantic model instance
document_type = DocumentType.model_validate_json(response.text)

print("Document Type:", document_type.document_type)
print("Confidence:", document_type.confidence)